In [1]:
import os
import itertools
import math
import numpy as np
import pandas as pd
import requests
import pickle
from datetime import datetime as dt
from datetime import timedelta
import pytz
from pytz import timezone
from pprint import pprint
import pandas_datareader.data as web
from collections import namedtuple
from operator import itemgetter

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import fundamentalanalysis as fa
import streamlit as st

import pandas_datareader.data as web
import yahoo_fin.stock_info as si
from urllib.request import Request, urlopen
from html_table_parser.parser import HTMLTableParser 
import streamlit as st

from load_data import *

pd.options.plotting.backend = "plotly"


In [2]:
rf_rates = get_rf_data()
SPY_df = get_SPY_data()
SPY_info_df = get_SPY_info()
ticker_list = SPY_info_df.index.to_list()
sector_list = SPY_info_df['Sector'].unique()
first_date = SPY_df.iloc[0].name
last_date = SPY_df.iloc[-1].name
yr_ago = last_date - timedelta(days=365)
# TTM_ratios, ratios_data_report = load_TTM_ratios()

In [3]:
df = get_ticker_data('tsla')
df.iloc[100].name

Timestamp('2010-11-18 00:00:00')

In [4]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
current = table[0]
hist = table[1]
hist

Date  Added                                Removed  \
                  Date Ticker                       Security  Ticker   
0      January 5, 2023    NaN                            NaN     VNO   
1      January 4, 2023   GEHC                  GE HealthCare     NaN   
2    December 22, 2022   STLD                 Steel Dynamics    ABMD   
3    December 19, 2022   FSLR                    First Solar    FBHS   
4    December 19, 2022    NaN                            NaN     MBC   
..                 ...    ...                            ...     ...   
315       June 9, 1999    WLP                      Wellpoint     HPH   
316  December 11, 1998    FSR                        Firstar     LDW   
317  December 11, 1998    CCL                 Carnival Corp.     GRN   
318  December 11, 1998   CPWR                      Compuware     SUN   
319      June 17, 1997    CCI  Countrywide Credit Industries     USL   

                                     \
                           Security   
0              Vornado Realty Trust   
1                               NaN   
2                           Abiomed   
3    Fortune Brands Home & Security   
4                       MasterBrand   
..                              ...   
315        Harnischfeger Industries   
316                           Amoco   
317                      General Re   
318                      SunAmerica   
319                          USLife   

                                                Reason  
                                                Reason  
0                     Market capitalization change.[4]  
1    S&P 500 constituent General Electric spun off ...  
2    S&P 500 constituent Johnson & Johnson acquired...  
3                     Market capitalization change.[7]  
4                     Market capitalization change.[7]  
..                                                 ...  
315           Harnischfeger filed for bankruptcy.[241]  
316            British Petroleum purchased Amoco.[242]  
317      Berkshire Hathaway purchased General Re.[242]  
318                     AIG purchased SunAmerica.[242]  
319                          AIG acquired USLife.[243]  

[320 rows x 6 columns]

In [14]:
added = hist['Date'].join(hist['Added'])
added['inSPY'] = True
added['Date'] = pd.to_datetime(added['Date'])
added

,Date,Ticker,Security,inSPY
0,2023-01-05,NaN,NaN,True
1,2023-01-04,GEHC,GE HealthCare,True
2,2022-12-22,STLD,Steel Dynamics,True
3,2022-12-19,FSLR,First Solar,True
4,2022-12-19,NaN,NaN,True
...,...,...,...,...
315,1999-06-09,WLP,Wellpoint,True
316,1998-12-11,FSR,Firstar,True
317,1998-12-11,CCL,Carnival Corp.,True
318,1998-12-11,CPWR,Compuware,True


In [16]:
removed = hist['Date'].join(hist['Removed'])
removed['inSPY'] = False
removed['Date'] = pd.to_datetime(removed['Date'])
removed


,Date,Ticker,Security,inSPY
0,2023-01-05,VNO,Vornado Realty Trust,False
1,2023-01-04,NaN,NaN,False
2,2022-12-22,ABMD,Abiomed,False
3,2022-12-19,FBHS,Fortune Brands Home & Security,False
4,2022-12-19,MBC,MasterBrand,False
...,...,...,...,...
315,1999-06-09,HPH,Harnischfeger Industries,False
316,1998-12-11,LDW,Amoco,False
317,1998-12-11,GRN,General Re,False
318,1998-12-11,SUN,SunAmerica,False


In [33]:
hist = pd.concat([added, removed]).sort_values('Date', ascending=False)
hist.dropna(inplace=True)
hist.Ticker.value_counts()

OI      3
CBE     3
MXIM    3
UA      3
COV     3
       ..
MGM     1
PKG     1
BBBY    1
MUR     1
USL     1
Name: Ticker, Length: 513, dtype: int64

In [19]:
si.get_data('^IRX')

,open,high,low,close,adjclose,volume,ticker
1970-03-25,6.150,6.150,6.150,6.150,6.150,0.0,^IRX
1970-03-26,6.260,6.260,6.260,6.260,6.260,0.0,^IRX
1970-03-27,NaN,NaN,NaN,NaN,NaN,NaN,^IRX
1970-03-29,NaN,NaN,NaN,NaN,NaN,NaN,^IRX
1970-03-30,6.320,6.320,6.320,6.320,6.320,0.0,^IRX
...,...,...,...,...,...,...,...
2023-01-22,NaN,NaN,NaN,NaN,NaN,NaN,^IRX
2023-01-23,4.533,4.543,4.525,4.535,4.535,0.0,^IRX
2023-01-24,4.575,4.578,4.568,4.573,4.573,0.0,^IRX
2023-01-25,4.563,4.565,4.545,4.545,4.545,0.0,^IRX


In [107]:
securities_list = fa.available_companies('a812dc7d35321ed0502ea50d8dc71be9')

In [108]:
len(securities_list)

67339

In [5]:
securities_list['type'].unique()

array(['etf', 'stock', 'trust', 'fund'], dtype=object)

In [6]:
securities_list[securities_list['type'] == 'etf']

,name,price,exchange,exchangeShortName,type
symbol,,,,,
SPY,SPDR S&P 500 ETF Trust,396.91,New York Stock Exchange Arca,AMEX,etf
GDX,VanEck Gold Miners ETF,24.19,New York Stock Exchange Arca,AMEX,etf
EEM,iShares MSCI Emerging Markets ETF,38.25,New York Stock Exchange Arca,AMEX,etf
XLF,Financial Select Sector SPDR Fund,33.74,New York Stock Exchange Arca,AMEX,etf
EFA,iShares MSCI EAFE ETF,60.49,New York Stock Exchange Arca,AMEX,etf
...,...,...,...,...,...
ASOL.SW,21Shares Solana staking ETP,19.08,Swiss Exchange,SIX,etf
AVAX.SW,21Shares Avalanche ETP,3.63,Swiss Exchange,SIX,etf
ABNB.SW,21Shares Binance BNB ETP,20.12,Swiss Exchange,SIX,etf


## Technical Analysis

In [22]:
def isSupport(df, i):
    '''Returns True if value is a price support level'''

    X = df['Low']
    support = X[i] < X[i - 1] \
                and X[i] < X[i + 1] \
                and X[i + 1] < X[i + 2] \
                and X[i - 1] < X[i - 2]

    return support


def isResistance(df, i):
    '''Returns True if value is a price resistance level'''

    X = df['High']
    resistance = X[i] > X[i - 1] \
                    and X[i] > X[i + 1] \
                    and X[i + 1] > X[i + 2] \
                    and X[i - 1] > X[i - 2] 

    return resistance


def sr_levels(df, start, end):
    '''Returns key support/resistance levels for a security'''

    df = df[start : end]
    df.columns = df.columns.str.title()
    support = []
    resistance = []
    s = (df['High'] - df['Low']).mean()

    def isFarFromLevel(l):
        '''
        Given a price value, returns False 
        if it is near some previously discovered key level
        '''
        
        levels = support + resistance
        return np.sum([abs(l - x[1]) < s for x in levels]) == 0

    for i in range(2, df.shape[0]-2):
        sprev = start if not support else df.iloc[support[-1][0]].name
        rprev = start if not resistance else df.iloc[resistance[-1][0]].name
        ix = df.iloc[i].name

        if isSupport(df, i):
            l = df['Low'][i]
            if isFarFromLevel(l):
                d = pd.Timedelta(ix - sprev).days
                support.append((i, l, d))
                df.loc[ix:, 'Support'] = l
                print(f'Support: {l} formed on {ix.date()} took {d} days to form')
        elif isResistance(df, i):
            l = df['High'][i]
            if isFarFromLevel(l):
                d = pd.Timedelta(ix - rprev).days
                resistance.append((i, l, d))
                df.loc[ix:, 'Resistance'] = l
                print(f'Resistance: {l} formed on {ix.date()} took {d} days to form')

    return support, resistance, df


In [23]:
def calculate_trends(df, start, end, time_frame, short_ma, inter_ma, primary_ma):
    df = df[start : end]
    df.columns = df.columns.str.title()  

    if time_frame != 'Daily':
        if time_frame == 'Weekly':
            fmt = 'W-MON'
        elif time_frame == 'Monthly':
            fmt = 'BMS'

        ix = df.asfreq(fmt).index
        open_ = df['Open'].resample(fmt).first()
        close = df['Close'].resample(fmt).last()
        high = df['High'].resample(fmt).max()
        low = df['Low'].resample(fmt).min()
        volume = df['Volume'].resample(fmt).sum()
        d = dict(Open=open_, High=high, Low=low, Close=close, Volume=volume)
        df = pd.DataFrame(d, index=ix)

    support, resistance, df = sr_levels(df, start, end)
    df[f'MA{short_ma}'] = df['Close'].rolling(short_ma).mean()
    df[f'MA{inter_ma}'] = df['Close'].rolling(inter_ma).mean()
    df[f'MA{primary_ma}'] = df['Close'].rolling(primary_ma).mean()
    levels = itertools.zip_longest(support, resistance)

    # for s,r in levels:
    #     print(s,r)

    return df

In [24]:
df = get_ticker_data('tsla')
end = dt.now()
years = 3
start = end - timedelta(365 * years)
time_frame, short_ma, inter_ma, primary_ma = 'Daily', 20, 50, 100

In [25]:
# pd.set_option('display.max_rows', None)
support, resistance, ta_df = sr_levels(df, start, end)
ta_df.iloc[short_ma:]

Resistance: 64.59933471679688 formed on 2020-02-04 took 6 days to form
Support: 45.79999923706055 formed on 2020-02-06 took 8 days to form
Support: 23.367332458496094 formed on 2020-03-18 took 41 days to form
Resistance: 95.3000030517578 formed on 2020-07-07 took 154 days to form
Resistance: 119.66600036621094 formed on 2020-07-13 took 6 days to form
Resistance: 167.49667358398438 formed on 2020-09-01 took 50 days to form
Resistance: 153.97999572753906 formed on 2020-09-15 took 14 days to form
Support: 139.68333435058594 formed on 2020-10-20 took 216 days to form
Resistance: 202.6000061035156 formed on 2020-11-30 took 76 days to form
Resistance: 218.10667419433597 formed on 2020-12-09 took 9 days to form
Support: 188.77999877929688 formed on 2020-12-10 took 51 days to form
Resistance: 231.6666717529297 formed on 2020-12-18 took 9 days to form
Support: 260.0333251953125 formed on 2021-01-29 took 50 days to form
Resistance: 292.5899963378906 formed on 2021-02-08 took 52 days to form
Resi

,Open,High,Low,Close,Adjclose,Volume,Ticker,Resistance,Support
2020-02-27,48.666668,49.318001,44.599998,45.266666,45.266666,364158000,TSLA,64.599335,45.799999
2020-02-28,41.980000,46.034668,40.768002,44.532665,44.532665,363343500,TSLA,64.599335,45.799999
2020-03-02,47.417332,49.579334,45.778000,49.574669,49.574669,302925000,TSLA,64.599335,45.799999
2020-03-03,53.666668,53.798668,47.740665,49.700668,49.700668,386760000,TSLA,64.599335,45.799999
2020-03-04,50.930668,51.101334,48.315334,49.966667,49.966667,225735000,TSLA,64.599335,45.799999
...,...,...,...,...,...,...,...,...,...
2023-01-20,128.679993,133.509995,127.349998,133.419998,133.419998,138429900,TSLA,280.786682,108.239998
2023-01-23,135.869995,145.380005,134.270004,143.750000,143.750000,203119200,TSLA,280.786682,108.239998
2023-01-24,143.000000,146.500000,141.100006,143.889999,143.889999,158699100,TSLA,280.786682,108.239998
2023-01-25,141.910004,146.410004,138.070007,144.429993,144.429993,192734300,TSLA,280.786682,108.239998


In [26]:
ta_df = calculate_trends(df, start, end, time_frame, short_ma, inter_ma, primary_ma)
ta_df.tail()

Resistance: 64.59933471679688 formed on 2020-02-04 took 6 days to form
Support: 45.79999923706055 formed on 2020-02-06 took 8 days to form
Support: 23.367332458496094 formed on 2020-03-18 took 41 days to form
Resistance: 95.3000030517578 formed on 2020-07-07 took 154 days to form
Resistance: 119.66600036621094 formed on 2020-07-13 took 6 days to form
Resistance: 167.49667358398438 formed on 2020-09-01 took 50 days to form
Resistance: 153.97999572753906 formed on 2020-09-15 took 14 days to form
Support: 139.68333435058594 formed on 2020-10-20 took 216 days to form
Resistance: 202.6000061035156 formed on 2020-11-30 took 76 days to form
Resistance: 218.10667419433597 formed on 2020-12-09 took 9 days to form
Support: 188.77999877929688 formed on 2020-12-10 took 51 days to form
Resistance: 231.6666717529297 formed on 2020-12-18 took 9 days to form
Support: 260.0333251953125 formed on 2021-01-29 took 50 days to form
Resistance: 292.5899963378906 formed on 2021-02-08 took 52 days to form
Resi

,Open,High,Low,Close,Adjclose,Volume,Ticker,Resistance,Support,MA20,MA50,MA100
2023-01-20,128.679993,133.509995,127.349998,133.419998,133.419998,138429900,TSLA,280.786682,108.239998,121.334000,154.794000,204.0394
2023-01-23,135.869995,145.380005,134.270004,143.750000,143.750000,203119200,TSLA,280.786682,108.239998,121.642999,153.843000,202.6287
2023-01-24,143.000000,146.500000,141.100006,143.889999,143.889999,158699100,TSLA,280.786682,108.239998,122.569999,153.169000,201.2906
2023-01-25,141.910004,146.410004,138.070007,144.429993,144.429993,192734300,TSLA,280.786682,108.239998,123.633999,152.243199,199.9788
2023-01-26,159.964996,161.419998,154.759995,160.270004,160.270004,233489389,TSLA,280.786682,108.239998,126.192499,151.529200,198.8099


In [11]:
def plot_trends(df, start, end, time_frame, short_ma, inter_ma, primary_ma):
    '''
    Returns candlestick chart with support/resistance levels
    and market cycle trend lines

    Parameters
    ----------
    df: DataFrame of a security's market data
    start: Start Date
    end: End Date
    time_frame: [Daily, Weekly, Monthly]
    short_ma: moving average window
    inter_ma: moving average window
    primary_ma: moving average window
    '''

    df = calculate_trends(df, start, end, time_frame, short_ma, inter_ma, primary_ma)
    name = df['Ticker'][0] if 'Ticker' in df.columns else ''
    cname = SPY_info_df.loc[name, 'Security']
    title1 = f'{cname}' # {time_frame} Trends & Support-Resistance Levels <br>
    title2 = ''

    fig = make_subplots(rows=2, cols=1,
                        shared_xaxes=True, 
                        vertical_spacing=0.01,
                        subplot_titles=(title1, title2), 
                        row_width=[0.2, 0.7])
    cs = go.Candlestick(x=df.index, 
                        open=df['Open'], 
                        high=df['High'],
                        low=df['Low'], 
                        close=df['Close'],
                        name=name)
    sma1 = go.Scatter(x=df.index, 
                      y=df[f'MA{short_ma}'], 
                      name=f'Short-Term Trend ({short_ma})',
                      line_width=1,
                      line_color='orange',
                      )
    sma2 = go.Scatter(x=df.index,
                      y=df[f'MA{inter_ma}'],
                      name=f'Intermediate Trend ({inter_ma})',
                      line_width=1.25)
    sma3 = go.Scatter(x=df.index,
                      y=df[f'MA{primary_ma}'],
                      name=f'Primary Trend ({primary_ma})',
                      line_width=1.5)
    
    fig.add_traces(data=[cs, sma1, sma2, sma3], rows=[1, 1, 1, 1], cols=[1, 1, 1, 1])   

    support, resistance = sr_levels(df, start, end)
    levels = support + resistance
    # Add support & resistance lines
    for i, l in levels:
        n = df.shape[0] - i
        fig.add_scatter(x=df.index[i:],
                        y=[l] * n,
                        name='S-R',
                        line={'color': 'yellow', 'width': 0.5},
                        mode='lines',
                        showlegend=False)
        
    # Volume subplot
    fig.add_trace(go.Bar(x=df.index, y=df['Volume'], name='Volume'),
                  row=2, col=1)
    fig.update_xaxes(showgrid=False, gridcolor='#BEBEBE')              
    fig.update_yaxes(showgrid=False)
    fig.layout.annotations[0].update(x=0.015)
    fig.layout.xaxis.rangeslider.visible = False

    return fig